In [2]:
%%javascript
// Making sure the outputs display correctly
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
# Importing required packages for this homework
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
#from IPython.display import HTML, display
#import tabulate
%matplotlib inline

def rounds(item):
    ret = float('%.2f'% (item * 100/100))
    return(ret)

path = os.getcwd()
print(path)

C:\Users\butle175\Downloads


In [4]:
# https://www.grc.nasa.gov/WWW/k-12/airplane/atmosmet.html
class atmosphere:
    def __init__(self, val, valGiven = 0,units = "SI"):
        #Convert from US to SI
        if units != "SI" and valGiven == 0:
            val = val / 3.281
        if units != "SI" and valGiven == 1:
            val = val * 0.04788
        #0 implies the given value is an altitude
        #1 implies the given value is a pressure
        if valGiven == 0:
            self.h = val
        elif valGiven == 1:
            self.P = val
        else:
            print("Not a valid 'valGiven' parameter.")
    def hCalc(self):
        if self.h < 11000:
            self.T = 15.04 - 0.00649*self.h #K
            self.P = 101.29 * ((self.T + 273.1)/288.08)**(5.256) #kPa
        elif self.h < 25000:
            self.T = -56.46
            self.P = 22.65 * np.exp(1.73 - 0.000157*self.h)
        elif self.h > 24999:
            self.T = -131.21 + 0.00299*self.h
            self.P = 2.488 * ((self.T + 273.1)/216.6)**(-11.388)
        self.rho = self.P / (0.2869 * (self.T + 273.1))
    def PCalc(self):
        if self.P > 22.632:
            self.T = (288.08*(self.P/101.29)**(1/5.256))-273.1
            self.h = (self.T - 15.04)/(-0.00649)
        elif self.P > 0.1113586:
            self.T = -56.46
            self.h = (1.73 - np.log((self.P/22.65)))/(0.000157)
        else:
            self.T = (216.6*(self.P/2.488)**(1/(-11.388)))-273.1
            self.h = (self.T + 131.21)/0.00299
        self.rho = self.P / (0.2869 * (self.T + 273.1))

In [33]:
#Finding angle it is falling at
angRad = 195 #pixels
angX = 124 #pixels
theta = np.arccos(124/195)
#print(rounds(theta),'radians')
#From flat going down
thetaFlat = np.pi - theta - np.pi/2
#print(rounds(thetaFlat))

Afin = (17.537 * 0.0254 ** 2)*np.cos(thetaFlat) #sq in

radRock = (8.5 / (np.pi * 2)) * 0.0254 #m
print("diameter:",(radRock*2)) #meters
lenNose = 9 * 0.0254 #m
lenBody = 30 * 0.0254 #m
Aflat = (radRock**2 * np.pi)+(radRock*2*lenBody)
Abody = Aflat * np.cos(thetaFlat) #sq in

totAreaFalling = Abody + Afin*2
print('falling area (US):', totAreaFalling)
print('Falling area:',totAreaFalling,'sq meters')
print((11*0.0254)**2*np.pi)

diameter: 0.0687231044270804
falling area (US): 0.06074218991561928
Falling area: 0.06074218991561928 sq meters
0.24524641988318885


In [34]:
## Cd with parachute
CdPara = (2 * 4.44822) / (1.229 * (0.06074 + ((11*0.0254)**2*np.pi)) * (1.8288**2))
print(CdPara)

7.073425647864539


In [38]:
#Quadratic regression
def quadReg(points):
    n = len(points)
    X = [item[0] for item in points]
    Y = [item[1] for item in points]
    Sx = sum(X)
    Sy = sum(Y)
    Sxx = sum([item**2 for item in X])
    Sxxx = sum([item**3 for item in X])
    Sxxxx = sum([item**4 for item in X])
    Sxy = sum([X[i] * Y[i] for i in range(n)])
    Sxxy = sum([X[i]**2 * Y[i] for i in range(n)])
    A = [[Sxxxx,Sxxx,Sxx],[Sxxx,Sxx,Sx],[Sxx,Sx,n]]
    b = [[Sxxy],[Sxy],[Sy]]
    coef = np.linalg.solve(A, b)
    return(coef)
#Reynolds numbers for alts in meters
Rey0 = [[(0+459.67)*(5/9),7900],[(22+459.67)*(5/9),7200],
        [(40+459.67)*(5/9),6800],[(60+459.67)*(5/9),6200],
        [(80+459.67)*(5/9),5800],[(100+459.67)*(5/9),5400]]
alt0 = quadReg(Rey0)
print(alt0)

[[ 6.50553685e-02]
 [-3.14758689e+01]
 [ 7.89483828e+03]]


In [52]:
#Coefficient of friction values (Turbulant)
Cf = [[1300000,0.006],[2000000,0.0056]]
m = (Cf[1][1]-Cf[0][1])/(Cf[1][0]-Cf[0][0])
b = Cf[0][1] - (m*Cf[0][0])
print(m,b)

-5.714285714285717e-10 0.006742857142857143


In [42]:
#Wet surface area to Body surface area
SwSbtNose = 8.6
SwSbtBody = 44.
SwSbt = SwSbtNose + SwSbtBody
print(SwSbt)

52.6


In [56]:
#####################################
##       START OF CODE FILE        ##
#####################################

##### DON'T CHANGE THESE #####
#Wet surface area to body surface area
SwSbt = 52.6
# Length over radius
Ld = 39/2.705634
# Length of the rocket
L = 39/12 #feet

##### CHANGE THIS ON DAY #####
T = 271 #kelvin

##### REGRESSIONS AND FUNCTIONS #####
#Quadratic Regression for Determining Reynolds Number
## Temperatures are in Kelvin
## Re = aT^2 + bT + c
a = 0.2107793
b = -164.31097
c = 36109.32
#Find the unit value of Reynolds from temprature
def reynolds(T,vel):
    ReUnit = a*T**2 + b*T + c
    Re = ReUnit * vel * L
    return(Re)

#Linear regression of coefficient of friction from Reynolds
## Take values directly from above
## Cf = m(Re) + b
def CfSolve(Re):
    #slope of regression
    m = -5.71429 * 10**(-10)
    #y-int of regressions
    b = 0.006742857
    Cf = m*Re + b
    return(Cf)

# Calculating Cd from these values
def Cd(T, vel):
    Rey = reynolds(T,vel)
    Cf = CfSolve(Rey)
    Cd = 1.02 * Cf * (1 + 
                      (1.5 / (Ld)**(3/2))
                     ) * SwSbt
    return(Cd)

'''
You can build a for loop around the Cd function and 
from there determine the coefficient of drag for every speed
you might find along the trajectory
'''

#Example case
testCase = Cd(T,125)
print("Coefficient of drag for 0 F and 125 m/s:\n",testCase)



Coefficient of drag for 0 F and 125 m/s:
 0.28133016717371057
